<a href="https://colab.research.google.com/github/Yangyu66666/Deep_learning/blob/main/site/zh-cn/tutorials/keras/%E6%96%87%E6%9C%AC%E5%88%86%E7%B1%BB_with_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# 使用 TensorFlow Hub 对电影评论进行文本分类

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/tutorials/keras/text_classification_with_hub"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>   </td>
  <td><a href="https://tfhub.dev/s?module-type=text-embedding"><img src="https://tensorflow.google.cn/images/hub_logo_32px.png">查看 TF Hub 模型</a></td>
</table>

此笔记本（notebook）使用评论文本将影评分为*积极（positive）*或*消极（nagetive）*两类。这是一个*二元（binary）*或者二分类问题，一种重要且应用广泛的机器学习问题。

本教程演示了使用 Tensorflow Hub 和 Keras 进行迁移学习的基本应用。

我们将使用包含 [Internet Movie Database](https://www.imdb.com/) 中的 50,000 条电影评论文本的 [IMDB 数据集](https://tensorflow.google.cn/api_docs/python/tf/keras/datasets/imdb)。先将这些评论分为两组，其中 25,000 条用于训练，另外 25,000 条用于测试。训练组和测试组是*均衡的*，也就是说其中包含相等数量的正面评价和负面评价。

此笔记本使用 [`tf.keras`](https://tensorflow.google.cn/guide/keras)（一个在 TensorFlow 中用于构建和训练模型的高级 API）和 [`tensorflow_hub`](https://tensorflow.google.cn/hub)（一个用于在单行代码中从 [TFHub](https://tfhub.dev) 加载训练模型的库）。有关使用 `tf.keras` 的更高级的文本分类教程，请参阅 [MLCC 文本分类指南](https://developers.google.com/machine-learning/guides/text-classification/)。

In [1]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

In [2]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
#Eager Execution（即即时执行模式）是 TensorFlow 2.x 中的一个执行模式。
#在Eager Execution模式下，TensorFlow会立即执行每个操作，而不需要构建整个计算图并在会话（Session）中运行。
#这使得 TensorFlow 的编程方式更加直观和灵活，让开发者可以像使用 NumPy 等其他库一样进行操作，并且可以即时查看操作结果，方便调试和开发。
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")
#检查系统中是否有可用的 GPU 设备。如果返回一个非空列表，则表示 GPU 可用；否则，GPU 不可用

Version:  2.12.0
Eager mode:  True
Hub version:  0.14.0
GPU is NOT AVAILABLE


## 下载 IMDB 数据集

[IMDB 评论](https://github.com/tensorflow/datasets)或 [TensorFlow Datasets](https://tensorflow.google.cn/datasets) 上提供了 IMDB 数据集。以下代码可将 IMDB 数据集下载到您的机器（或 Colab 运行时）上：

In [3]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'), #前百分之60和百分之60之后
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteEDXC39/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteEDXC39/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteEDXC39/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## 探索数据

我们花一点时间来了解数据的格式。每个样本都是一个代表电影评论的句子和一个相应的标签。句子未经过任何预处理。标签是一个整数值（0 或 1），其中 0 表示负面评价，1 表示正面评价。

我们来打印下前十个样本。

In [8]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
#通过调用 train_data.batch(10)，我们将训练数据集 train_data 按照 batch 大小 10 进行分批，得到一个大小为 10 的 batch
train_examples_batch[1]
#我们使用iter()创建了一个迭代器，并通过next(...)函数获取迭代器的下一个元素，也就是一个batch的数据。由于每次迭代器只返回一个batch的数据，因此这里取出的是一个大小为10的 batch

<tf.Tensor: shape=(), dtype=string, numpy=b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.'>

我们再打印下前十个标签。

In [7]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

## 构建模型

神经网络由堆叠的层创建而成，这需要从三个主要方面来进行体系结构决策：

- 如何表示文本？
- 在模型中使用多少个层？
- 为每个层使用多少个*隐藏单元*？

本示例中，输入数据由句子组成。预测的标签为 0 或 1。

表示文本的一种方式是将句子转换为嵌入向量。使用一个预训练文本嵌入向量作为首层，这样做有三个优点：

- 不必担心文本预处理
- 可以从迁移学习中受益
- 嵌入向量具有固定大小，更易于处理

在本示例中，您使用来自 [TensorFlow Hub](https://tfhub.dev) 的 **预训练文本嵌入向量模型**，名称为 [google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2)。

本教程中还可以使用来自 TFHub 的许多其他预训练文本嵌入向量：

- [google/nnlm-en-dim128/2](https://tfhub.dev/google/nnlm-en-dim128/2) - 基于与 [google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2) 相同的数据并使用相同的 NNLM 架构进行训练，但具有更大的嵌入向量维度。更大维度的嵌入向量可以改进您的任务，但可能需要更长的时间来训练您的模型。
- [google/nnlm-en-dim128-with-normalization/2](https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2) - 与 [google/nnlm-en-dim128/2](https://tfhub.dev/google/nnlm-en-dim128/2) 相同，但具有额外的文本归一化，例如移除标点符号。如果您的任务中的文本包含附加字符或标点符号，这会有所帮助。
- [google/universal-sentence-encoder/4](https://tfhub.dev/google/universal-sentence-encoder/4) - 一个可产生 512 维嵌入向量的大得多的模型，使用深度平均网络 (DAN) 编码器训练。

还有很多！在 TFHub 上查找更多[文本嵌入向量模型](https://tfhub.dev/s?module-type=text-embedding)。

让我们首先创建一个使用 Tensorflow Hub 模型嵌入（embed）语句的Keras层，并在几个输入样本中进行尝试。请注意无论输入文本的长度如何，嵌入（embeddings）输出的形状都是：`(num_examples, embedding_dimension)`。

In [16]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)
#hub_layer(train_examples_batch[:3])
hub_layer(train_examples_batch[2:3])

<tf.Tensor: shape=(1, 50), dtype=float32, numpy=
array([[ 0.7456154 ,  0.21256861,  0.14400336,  0.5233862 ,  0.11032254,
         0.00902788, -0.3667802 , -0.08938274, -0.24165542,  0.33384594,
        -0.11194605, -0.01460047, -0.0071645 ,  0.19562712,  0.00685216,
        -0.24886718, -0.42796347,  0.18620004, -0.05241098, -0.66462487,
         0.13449019, -0.22205497,  0.08633006,  0.43685386,  0.2972681 ,
         0.36140734, -0.7196889 ,  0.05291241, -0.14316116, -0.1573394 ,
        -0.15056328, -0.05988009, -0.08178931, -0.15569411, -0.09303783,
        -0.18971172,  0.07620788, -0.02541647, -0.27134508, -0.3392682 ,
        -0.10296468, -0.27275252, -0.34078008,  0.20083304, -0.26644835,
         0.00655449, -0.05141488, -0.04261917, -0.45413622,  0.20023568]],
      dtype=float32)>

现在让我们构建完整模型：

In [10]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


层按顺序堆叠以构建分类器：

1. 第一层是 TensorFlow Hub 层。该层使用预训练的 SavedModel 将句子映射到其嵌入向量。您使用的预训练文本嵌入向量模型 ([google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2)) 可将句子拆分为词例，嵌入每个词例，然后组合嵌入向量。生成的维度是：`(num_examples, embedding_dimension)`。对于此 NNLM 模型，`embedding_dimension` 为 50。
2. 该定长输出向量通过一个有 16 个隐层单元的全连接层（`Dense`）进行管道传输。
3. 最后一层与单个输出结点紧密相连。

我们来编译模型。

### 损失函数与优化器

一个模型需要一个损失函数和一个优化器来训练。由于这是一个二元分类问题，且模型输出 logits（具有线性激活的单一单元层），因此，我们将使用 `binary_crossentropy` 损失函数。

这并非损失函数的唯一选择，例如，您还可以选择 `mean_squared_error`。但是，一般来说，`binary_crossentropy` 更适合处理概率问题，它可以测量概率分布之间的“距离”，或者在我们的用例中，是指真实分布与预测值之间的差距。

稍后，当您探索回归问题（例如，预测房屋价格）时，您将看到如何使用另一个称为均方误差的损失函数。

现在，配置模型来使用优化器和损失函数：

In [11]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## 训练模型

使用包含 512 个样本的 mini-batch 对模型进行 10 个周期的训练，也就是在 `x_train` 和 `y_train` 张量中对所有样本进行 10 次迭代。在训练时，监测模型在验证集的 10,000 个样本上的损失和准确率：

In [12]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)
#train_data.shuffle(10000):这一步首先对训练数据集train_data进行随机打乱（shuffle）。参数10000表示打乱时使用的随机种子.
#打乱数据可以增加样本的随机性，有助于模型更好地泛化。打乱后的数据将用于训练。
#batch(512): 接着，我们通过batch(512)将打乱后的训练数据集分成大小为512的batch。这样可以将训练数据分批输入模型进行梯度下降更新参数，有助于提高训练效率和减少内存消耗。

Epoch 1/10
30/30 [==============================] - 44s 1s/step - loss: 0.6597 - accuracy: 0.5465 - val_loss: 0.5999 - val_accuracy: 0.6089
Epoch 2/10
30/30 [==============================] - 50s 2s/step - loss: 0.5294 - accuracy: 0.7104 - val_loss: 0.4912 - val_accuracy: 0.7556
Epoch 3/10
30/30 [==============================] - 63s 2s/step - loss: 0.3991 - accuracy: 0.8294 - val_loss: 0.3998 - val_accuracy: 0.8138
Epoch 4/10
30/30 [==============================] - 43s 1s/step - loss: 0.2905 - accuracy: 0.8885 - val_loss: 0.3453 - val_accuracy: 0.8438
Epoch 5/10
30/30 [==============================] - 45s 2s/step - loss: 0.2128 - accuracy: 0.9258 - val_loss: 0.3204 - val_accuracy: 0.8659
Epoch 6/10
30/30 [==============================] - 43s 1s/step - loss: 0.1578 - accuracy: 0.9507 - val_loss: 0.3074 - val_accuracy: 0.8660
Epoch 7/10
30/30 [==============================] - 42s 1s/step - loss: 0.1157 - accuracy: 0.9683 - val_loss: 0.3049 - val_accuracy: 0.8701
Epoch 8/10
30/30 [==

## 评估模型

我们来看一下模型的性能如何。将返回两个值。损失值（一个表示误差的数字，值越低越好）与准确率。

In [13]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 9s - loss: 0.3646 - accuracy: 0.8481 - 9s/epoch - 185ms/step
loss: 0.365
accuracy: 0.848


这种相当简单的方法可以达到约 87% 的准确率。使用更高级的方法，模型的准确率应该会接近 95%。

## 延伸阅读

- 有关处理字符串输入的更通用方式以及对训练过程中准确率和损失进度的更详细分析，请参阅[使用预处理文本的文本分类](./text_classification.ipynb)教程。
- 尝试更多使用来自 TFHub 的训练模型的[文本相关教程](https://tensorflow.google.cn/hub/tutorials#text-related-tutorials)。